In [3]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../data/Motor_Vehicle_Collisions.csv')

In [4]:
# Convert LATITUDE / LONGITUDE to float 
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'].str.replace(',', '.'), errors='coerce')
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'].str.replace(',', '.'), errors='coerce')

# Check how many are missing
missing_coords = df[['LATITUDE','LONGITUDE']].isna().sum()
print("Missing coordinates:\n", missing_coords)


Missing coordinates:
 LATITUDE     23696
LONGITUDE    23696
dtype: int64


In [ ]:
geolocator = Nominatim(user_agent="nyc_collisions")

def geocode_address(row):
    if pd.isna(row['LATITUDE']) or pd.isna(row['LONGITUDE']):
        try:
            address = f"{row['ON STREET NAME']} & {row['CROSS STREET NAME']}, {row['BOROUGH']}, NYC"
            location = geolocator.geocode(address)
            if location:
                return pd.Series([location.latitude, location.longitude])
        except:
            return pd.Series([np.nan, np.nan])
    else:
        return pd.Series([row['LATITUDE'], row['LONGITUDE']])

# Apply geocoding only to missing rows
df[['LATITUDE','LONGITUDE']] = df.apply(geocode_address, axis=1)
